In [1]:
from model import MusicTransformer
from custom.layers import *
from custom import callback
from tensorflow.python import keras
import params as par
from tensorflow.python import enable_eager_execution
from tensorflow.python.keras.optimizer_v2.adam import Adam
from data import Data
import utils
tf.executing_eagerly()

True

Ściągnięcie zbiorów uczących (skrypt linuxowy, można też samemy podać jakiś inny zbiór):  
sh dataset/script/{ecomp_piano_downloader, midi_world_downloader, ...}.sh  
(np. sh dataset/script/classic_piano_dataset.sh)  
  
Preprocessing (pliki mmuzyczne są konwertowane do sekwencji liczb i zapisywane za pomocą pickle):  
python preprocess.py {midi_load_dir} {dataset_save_dir}  
(np. python preprocess.py "dataset\scripts\classic_piano_dataset"   "dataset\scripts\classic_piano_dataset_preprocessed")  
  
(jeśli skrypt chce się uruchomić prosto z notebooka to wystarczy przed komendą shellową wpisać "!")  

Uczenie:  
  
python train.py --epochs={NUM_EPOCHS} --load_path={NONE_OR_LOADING_DIR} --save_path={SAVING_DIR} --max_seq={SEQ_LENGTH} --pickle_dir={DATA_PATH} --batch_size={BATCH_SIZE} --l_r={LEARNING_RATE}  
  
(Przykładowo:  
python train.py --epochs=300 --save_path="sample_model"   
--max_seq=200 --pickle_dir="dataset\scripts\classic_piano_dataset_preprocessed" --batch_size=4)  

W pliku "sample_model" znajduje się wstępnie nauczona przeze mnie sieć nauronowa (max_seq=500, batch=3, epochs=400 ~= 4 godziny uczenia). Aby sprawdzić jej działanie:

In [22]:
from model import MusicTransformerDecoder
opt = Adam(par.l_r)
mt = MusicTransformerDecoder(
  	embedding_dim=256, vocab_size=par.vocab_size, 
  	num_layer=6, 
  	max_seq=500,
  	dropout=0.1,
  	debug=False,
    loader_path = r"sample_model"
)
# mt.compile(optimizer=opt, loss=callback.TransformerLoss())

In [29]:
output = mt.generate(prior=[70], length=2048)
output

array([ 70, 278, 368,  67, 280, 195, 368,  69, 267, 197, 368,  71, 267,
       199, 369,  72, 267, 200, 369,  74, 267, 202, 370,  76, 268, 204,
       366,  54, 371,  78, 303, 182, 206, 369,  43, 372,  79, 279, 171,
       207, 368,  69, 366,  65, 279, 197, 193, 368,  67, 366,  64, 279,
       195, 192, 368,  66, 366,  62, 279, 194, 190, 368,  67, 366,  64,
       279, 195, 192, 368,  66, 366,  62, 279, 194, 190, 368,  67, 366,
        64, 279, 195, 192, 368,  66, 366,  62, 279, 194, 190, 368,  67,
       366,  64, 281, 195, 192, 368,  66, 366,  62, 280, 194, 190, 368,
        67, 366,  64, 280, 195, 192, 370,  71, 366,  65, 280, 199, 193,
       369,  69, 366,  65, 280, 197, 193, 368,  67, 366,  64, 279, 195,
       192, 369,  77, 366,  65, 279, 205, 193, 368,  76, 366,  64, 280,
       204, 192, 369,  75, 366,  65, 280, 203, 193, 368,  74, 366,  65,
       280, 202, 193, 369,  72, 366,  64, 280, 200, 192, 368,  74, 366,
        65, 280, 202, 193, 369,  69, 366,  65, 280, 197, 193, 37

In [30]:
from  midi_processor.processor import decode_midi
decode_midi(output.tolist(), 'generated_2.mid')

info removed pitch: 80


In [ ]:
Jeśli chcesz przesłuchać stworzonej piosenki a nie posiadasz np. DAW (FL Studio), można użyć sekwencera online:
https://onlinesequencer.net/import
Uploaduje się piosenkę (.mid), akceptuje domyśle ustawienia i na ekranie pokazuje się widok piano roll (można też odtworzyć utwór).